In [6]:
from os import environ
from dotenv import load_dotenv
import pandas as pd
import boto3
from datetime import date
import psycopg2
from openai import OpenAI
import json
import logging
import time

In [10]:
def get_db_connection():
    """Returns a database connection."""
    try:
        return psycopg2.connect(
            host=environ["DB_HOST"],
            port=environ["DB_PORT"],
            database=environ["DB_NAME"],
            user=environ["DB_USER"],
            password=environ["DB_PASSWORD"]
            )
    except psycopg2.OperationalError as error:
        logging.exception("Error connecting to the database: %s", error)
        raise

In [40]:
def load_data():
    """Loads stories with greatest score change over last 24hrs from RDS.
    Returns them as a Dataframe object."""

    query = """
            SELECT
                records.story_id, 
                MAX(score) - MIN(score) AS score_change,
                stories.title,
                stories.story_url,
                MAX(record_time) AS latest_update
            FROM records
            JOIN stories ON records.story_id = stories.story_id
            WHERE record_time >= NOW() - INTERVAL '24 hours'
            GROUP BY records.story_id, stories.title, stories.story_url
            ORDER BY score_change 
                DESC
            ;
            """

    conn = get_db_connection()
    with conn.cursor() as curr:
        curr.execute(query)
        df = pd.DataFrame(curr.fetchall())
        df.columns = [desc[0] for desc in curr.description]

    return df

start_time = time.perf_counter()
df = load_data()
print(f"Runtime: {(time.perf_counter() - start_time) * 100}\n\n")
# print(df)

Runtime: 14.757779199862853




In [13]:
def get_url_list():
    df = load_data()
    return df['story_url'].to_list()

In [14]:
url_list = get_url_list()
url_list

['https://www.clicks.tech/',
 'https://www.robinsloan.com/notes/home-cooked-app/',
 'https://sonnet.io/posts/sit/',
 'https://commandcenter.blogspot.com/2024/01/what-we-got-right-what-we-got-wrong.html',
 'https://papereditor.app/dev',
 'https://nestflix.fun/',
 'https://gist.github.com/FeepingCreature/5dff669aad380a123b15659e195fb96c',
 'https://smallstep.com/blog/home-network-eap-tls-wifi/',
 'https://mashable.com/article/youtube-demontizes-public-domain-steamboat-willie-disney-copyright-claim',
 'https://people.inf.ethz.ch/wirth/ProjectOberon/PO.System.pdf',
 'https://www.ft.com/content/0a3c9806-8b0f-4cca-a4e5-e1e6dd6d395b',
 'https://www.reuters.com/business/healthcare-pharmaceuticals/consumer-reports-finds-widespread-presence-plastics-food-2024-01-04/',
 'http://www.danamania.com/print/Apple%20Pascal%20Poster/PascalPosterV3%20A1.pdf',
 'https://andykong.org/blog/icloudconfusion',
 'https://www.da.vidbuchanan.co.uk/blog/python-swar.html',
 'https://edwarddonner.com/2024/01/02/fine-

In [15]:
df = load_data()
df.head()

,story_id,score_change,title,story_url,latest_update
0,38871987,826,Clicks – Physical keyboard for iPhone,https://www.clicks.tech/,2024-01-05 19:03:18.831362
1,38877423,596,An app can be a home-cooked meal (2020),https://www.robinsloan.com/notes/home-cooked-app/,2024-01-05 19:03:18.831362
2,38869235,560,Sit.,https://sonnet.io/posts/sit/,2024-01-05 19:03:18.831362
3,38872362,528,"Go: What we got right, what we got wrong",https://commandcenter.blogspot.com/2024/01/wha...,2024-01-05 19:03:18.831362
4,38866170,452,9 years of Apple text editor solo dev,https://papereditor.app/dev,2024-01-05 19:03:18.831362


In [42]:
sort_by_score = df.sort_values(by='score', ascending=False)
top_5_stories = sort_by_score.head(5)


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/kevinchirayil/Documents/Coursework/TechNews/email_lambda/venv-email-lambda/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
  File "/var/folders/zh/2ptn55r53md865jjw0gf0sb00000gp/T/ipykernel_73885/193496872.py", line 1, in <module>
    sort_by_score = df.sort_values(by='score', ascending=False)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/kevinchirayil/Documents/Coursework/TechNews/email_lambda/venv-email-lambda/lib/python3.11/site-packages/pandas/core/frame.py", line 6944, in sort_values
  File "/Users/kevinchirayil/Documents/Coursework/TechNews/email_lambda/venv-email-lambda/lib/python3.11/site-packages/pandas/core/generic.py", line 1844, in _get_label_or_level_values
KeyError: 'score'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/kevinchirayil/Documents/Coursework/TechNews/email_lambda/venv

In [43]:
print( date.today().strftime("%B %d"))

January 05
